***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [87]:
# project group 16
# 212240741 323794131
# data retrieval
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-b296  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [88]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [89]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar 11 12:58 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [91]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [92]:
spark

In [93]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'works89651257' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:      
    if b.name != 'graphframes.sh' and  'docsss' not in b.name and  'postings_gcp' not in b.name and 'postings_new' not in b.name:
        paths.append(full_path+b.name)
        

In [94]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py
import inverted_index_gcp
from inverted_index_gcp import InvertedIndex
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

inverted_index_gcp.py


24/03/11 16:31:44 WARN SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


# Building an inverted index

In [ ]:
### functions and variables to be used ###
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
NUM_BUCKETS = 124
time
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, doc_id):
    # applaying stem
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
    ps = nltk.stem.PorterStemmer()
    tokens = [ps.stem(token.group()) for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]
    word_counts = Counter(tokens)
    posting_list = [(token, (doc_id, tf)) for token, tf in word_counts.items()]
    return posting_list



def reduce_word_counts(unsorted_pl):
    sorted_pl = sorted(unsorted_pl, key=lambda x: x[1])
    return sorted_pl



def calculate_df(postings):
    # Flatten the postings RDD and extract terms
    terms = postings.flatMap(lambda x: [x[0]] * len(x[1]))
    df_counts = terms.map(lambda term: (term, 1)).reduceByKey(lambda a, b: a + b)
    return df_counts



def partition_postings_and_write(postings_filtered):
    posting_grouped = postings_filtered.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()
    return posting_grouped.map(lambda x: InvertedIndex.write_a_posting_list(x, "postings_new", "bucket-get-started_x-signifier-416215"))


# tokenization
def tokenize_and_steem_word(text):
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
    return [token.group() for token in RE_WORD.finditer(text.lower())]

# all SW
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                "may", "first", "see", "history", "people", "one", "two", 
                "part", "thumb", "including", "second", "following", 
                "many", "however", "would", "became"]
all_stopwords = english_stopwords.union(corpus_stopwords)
# stemmer
ps = nltk.stem.PorterStemmer()

# index init variables update for calculations

inverted = InvertedIndex()
counter = 0
sum_list_of_avg_times = 350
len_list_of_avg_times = 1
postings_filtered_all = 0
total = 0
total_time = 0

for path in paths:
    x = time()
    data = {}
    data2 = {}
    counter += 1
    # title id dict update
    print(f'path {counter}: title id dict update')
    parquetFile = spark.read.parquet(path)
    # timer stuff
    len_parquet = int(parquetFile.count())
    estimated = len_parquet/(sum_list_of_avg_times/len_list_of_avg_times)
    print(f"estimated time for path {counter}: {estimated} Sec")
    doc_id_title = parquetFile.select("id", "title").rdd.map(lambda row: (row.id, row.title))
    data = doc_id_title.map(lambda x: (x[0], x[1])).collectAsMap()
    
    # doc_len update
    print(f'path {counter}: doc_len update')
    doc_id_len = parquetFile.select("id", "text").rdd.map(lambda row: (row.id, row.text))
    length_data = doc_id_len.map(lambda x: (x[0], len([word for word in tokenize_and_steem_word(x[1]) if word not in all_stopwords])))
    data2 = length_data.collectAsMap()
    
    ### Index update ###
    print(f'path {counter}: Index update')
    doc_text_pairs = parquetFile.select("text", "id").rdd
      
    
    # writing to the index
    print(f'path {counter}: prapering variables...')
    word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    postings_filtered = postings.filter(lambda x: len(x[1])>1)
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()
    if not postings_filtered_all == 0:
        postings_filtered_all = postings_filtered_all.union(postings_filtered).reduceByKey(lambda x, y: x + y)
    else:
        postings_filtered_all = postings_filtered
    '''_ = partition_postings_and_write(postings_filtered).collect()'''
    
    
    
    # finally update the index
    print(f'path {counter}: finally update the index...')
    inverted.df.update(w2df_dict)
    inverted.idtotitledict.update(data)
    inverted.doc_len.update(data2)
    print(f'path {counter} completed')
    y = time()
    total_time += (y - x)
    total += len_parquet
    sum_list_of_avg_times += total / total_time
    len_list_of_avg_times += 1
    print(f"Actuall Time: {(y - x)}\n Estimated Time: {estimated}")
    

    
    
print(f'All paths are completed!')

# collect all posting lists locations into one super-set
print(f'path {counter}: started writing PLs to index...')

_ = partition_postings_and_write(postings_filtered_all).collect()
print(f'path {counter}: collecting all posting lists locations into one super-set...')

for blob in client.list_blobs(bucket_name, prefix='postings_new'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            if k in inverted.posting_locs.keys():
                inverted.posting_locs[k].extend(v)
            else:
                inverted.posting_locs[k] = v
            
# update index in bucket
print(f'path {counter}: updating index in bucket...')
inverted.write_index('.', 'index')
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst
print('Index has been created !')
    
    
    
    

path 1: title id dict update


estimated time for path 1: 484.3142857142857 Sec


path 1: doc_len update


path 1: Index update
path 1: prapering variables...


path 1: finally update the index...
path 1 completed
Actuall Time: 1620.0642037391663
 Estimated Time: 484.3142857142857
path 2: title id dict update


...

path 15: Index update
path 15: prapering variables...


path 15: finally update the index...
path 15 completed
Actuall Time: 866.792387008667
 Estimated Time: 1066.4605356628517
path 16: title id dict update


estimated time for path 16: 1059.0637514161085 Sec


path 16: doc_len update


path 16: Index update
path 16: prapering variables...
